In [70]:
import math
import random
from time import perf_counter
import numpy as np
from scipy import optimize
from autograd.misc.optimizers import adam
import numdifftools as nd

In [71]:
io_values = [[(4.4793, -4.0765, -4.0765), 0], [(-4.1793, -4.9218, 1.7664), 1], [(-3.9429, -0.7689, 4.883), 1]]

In [72]:
#g(x) de la consigna
def g(x):
    try:
        return math.exp(x) / (1 + math.exp(x))
    except OverflowError:
        return 1

In [73]:
#Funcion dada por la consigna
def F(W, w, w0, E):
    return g(sum(
        W[j + 1] * g(sum(
            w[j][k] * E[k]
            for k in range(0, 3)
        ) - w0[j])
        for j in range(0, 2)
    ) - W[0])

In [74]:
#Error que se comete
def E(W, w, w0):
    return sum((OUT - F(W, w, w0, IN)) ** 2 for (IN, OUT) in io_values)

In [75]:
def E2(w,i=None,u=None):
    return E(w[:3],np.array([w[3:6],w[6:9]]),w[9:])

In [76]:
# x0 = np.array([random.uniform(0,1) for i in range(0,11)])
x0 = np.array([0 for i in range(0,11)])

In [77]:
#Gradiente descendiente
t0 = perf_counter()
result = optimize.minimize(E2,x0=x0,method='L-BFGS-B')
tf = perf_counter()
print(f'execution time: {(tf-t0)*1000}ms')
print(f'error: {result.fun}')
print(f'x: {result.x}')

execution time: 21.81369999971139ms
error: 5.141418296604767e-06
x: [ 6.33723342  6.61920307  6.61920307 -6.33923132  1.6121689   5.44763473
 -6.33923132  1.6121689   5.44763473  0.23743733  0.23743733]


In [78]:
#Gradientes conjugados
t0 = perf_counter()
result = optimize.minimize(E2,x0=x0,method='CG')
tf = perf_counter()
print(f'execution time: {(tf-t0)*1000}ms')
print(f'error: {result.fun}')
print(f'x: {result.x}')

execution time: 67.80409999964832ms
error: 4.720771989600081e-06
x: [ 6.14978479  7.12182314  7.12182314 -2.76091069  0.53929397  2.345937
 -2.76091069  0.53929397  2.345937    0.06283709  0.06283709]


In [79]:
#ADAM
t0 = perf_counter()

result = adam(nd.Gradient(E2),x0,step_size=0.80085)
tf = perf_counter()
print(f'execution time: {(tf-t0)*1000}ms')
print(f'error: {E2(result)}')
print(f'x: {result}')

execution time: 1720.3463000000738ms
error: 4.896332467001788e-06
x: [ 6.71560625  6.67608314  6.67608314 -5.08802692  2.77629052  4.76615146
 -5.08802692  2.77629052  4.76615146  2.77230361  2.77230361]
